# Diffusion math
This notebook provides bits and pieces of good insights to aid in understanding the math of diffusion models, in particular the math will be focused on [DDPM](https://arxiv.org/pdf/2006.11239) (Denoising diffusion probabilistic models).

## Outline
* Prequisites
* Diffusion informal walkthrough - noise formulation
* Diffusion formal walkthrough

I've split the math techniques section into a separate notebook. You can skip the math techniques section entirely if you are not interested in understanding the derivations of diffusion. It basically lists miscellaneous math tricks that you would encounter when reading the derivations, although it's not exhaustive. Furthermore, a full mastery of the prerequisites listed in prequisites section is not necessary, a vague understanding of the concepts is sufficient. The diffusion informal walkthrough is a section dedicated to give intuition on the diffusion process, highlighting the mathematical interpretations of different parts of the process, without giving the derivations. The derivations is dedicated to the last section diffusion formal walkthrough, in which I merely refer to other papers that have already outlined the them. One paper I highly recommend is the one by [Calvin Luo - Understanding Diffusion Models: A Unified Perspective](https://arxiv.org/abs/2208.11970).

## Prerequisites
General maturity in probability theory is good and basic familiarity with integrals. You don't necessarily need to know how to solve integrals, but be familiar with what they mean conceptually and how to interpret them. Otherwise, the really relevant concepts are
* Joint, conditional, marginal probability
* Expectation, variance, standard deviation, covariance
* Bayes' theorem
* Univariate and multivariate gaussian
* KL-divergence
* Information theory (self-information + entropy)
* Evidence lower bound (ELBO)
* Basic understanding of VAE helps alot since diffusion can be seen as an extension of VAE, but is not necessary

You can either google these or look at my notebooks for these concepts.
I cover the following concepts in [probability-fastai-probability-theory-concepts](probability-fastai-probability-theory-concepts.ipynb)
* Joint, conditional, marginal probability
* Expectation, variance, standard deviation, covariance
* Bayes' theorem
* Univariate and multivariate gaussian

And concepts below can be found in [probability-fastai-probabilistic-machine-learning-concepts](probability-fastai-probabilistic-machine-learning-concepts.ipynb)
* KL-divergence
* Information theory (self-information + entropy)
* Evidence lower bound (ELBO)

## Diffusion informal walkthrough - noise formulation
This section focuses more on the intuition rather than the rigorous parts of diffusion. The purpose is to tie together insights that we have collected as a group about diffusion into a coherent walkthrough.

**Outline**
* Intro
* Assumptions, data format and preprocessing
* Forward process
* Loss function
* Training
* Backward process

### Intro
Imagine we have an image dataset and we want to train a model to generate samples that are representative of the data images. How do we do it? There are lots of different methods nowadays, and they come with different flavours. I'm not going to cover any of the other methods, as I quite frankly don't remember them well enough to even provide a brief description, but I will include two images that shows a nice comparison of diffusion models and some of the other methods that are popular.

![](assets/advantage_genai_models.png)

This shows that diffusion models cover two of the features, high quality samples and mode coverage/diversity, while being slower to sample compared to GANs, VAEs and models using normalizing flows. The slower sampling has to do with the backward process in DDPM being markovian, thus not able to jump time steps and needing a large number of steps to converge to a clean image (hundreds to thousand for DDPM). The [DDIM](https://arxiv.org/pdf/2010.02502) paper however seems to speed this up by instead working with a non-markovian structure for the backward process, thus being able to jump time steps. There are other papers since DDPM and DDIM that has tried to improve the speed of sampling ($\textcolor{red}{\text{Insert popular references}}$). We also have this nice table to show general comparisons between them.

![](assets/genai_models_table_features.png)

Here likelihood basically means whether computing the likelihood p(x) for any given **data point** x is exact or not. Diffusion models only do this approximately. Also reconstruction is hard for a diffusion model since the diffusion paths vary alot even from the same starting point, so it's never guaranteed that you would end up with the exact same image starting from the exact same noise. As for the bottleneck the dimension is the same for input and output, so it never changes throughout the process, unlike VAE that has a bottleneck between the encoder and decoder after the dimensionality has been reduced. The mode covering/diversity feature of diffusion models is thanks to a multitude of noise sources (including the scheduler) and stochasticity being introduced during the forward and backward process. This has a positive effect on preventing mode collapse, since areas with low probability density in the data space will be assigned more probability or be explored more in the diffusion paths.  Also the training objective is a minimization of a KL-divergence that is a M-projection (forward KL-divergence) $D_{KL}(\text{true distribution} || \text{approximate distribution})$, meaning it's mode covering to begin with. 

Diffusion models take inspiration from bayesian approaches and since it's a markovian hierarchiecal VAE that means it's also using [VI (variational inference)](https://en.wikipedia.org/wiki/Variational_Bayesian_methods) just as VAE to optimize a variational lower bound (ELBO) in order to approximate the log marginal likelihood of the data by learning in this case the backward process. In addition to this it uses MLE for training at least in DDPM, which is interesting. We will show later on that for the noise formulation MSE of noise is used on the network as a loss function to learn the noise. Since MSE is equivalent to maximizing the log likelihood under a gaussian distribution with fixed variance, then in that sense MLE is used to train the model.
As for approximating the backward process, which you will see later on in the loss section, we will be indirectly matching $q_{\theta}(x_{t-1}|x_t)$ that is the posterior in the backward process to the true posterior $q(x_{t-1} | x_t, x_0)$ by training a model to predict noise. The form for the true posterior backward distribution can be derived by using Bayes' rule.

Anyways, since this notebook is about diffusion the following sections will give intuition on how diffusion in DDPM works. The brief overview is that even for diffusion models there are different formulations one can go with that leads to different loss functions for diffusion and in this section we go with the noise formulation, where the model has to learn the noise in a noisy image. This formulation seems to be the most popular on the internet when covering the basics of diffusion, mostly because the [original stable diffusion](https://arxiv.org/pdf/2112.10752), [GLIDE](https://arxiv.org/pdf/2112.10741) and [DALL-E 2](https://arxiv.org/pdf/2204.06125v1) used it.


### Assumptions, data format and preprocessing
Assumptions
* Data is i.i.d, meaning each sample comes from the same distribution and are independent. We call this distribution the data distribution $p(x)$.
* Forward and backward process follows a markov chain, i.e next latent is only dependent on previous latent
* Latent dimension and data dimension are equal throughout the forward and backward process
* Distribution of intermediate latents in the forward are gaussians, these gaussians are not learnt. This means the encoder part of diffusion is never learnt, it's pre-defined as gaussians together with a scheduler. 
* Distribution of latent at the end of forward process converges to standard gaussian when timesteps $t \to \infty$

Data format is 2D images but with color channel (greyscale only has 1 color channel) becomes 3D. With a batch this goes up to a 4D array, which is what you would encounter in practise. But in the formulas we are usually only thinking of a single image at a time, and that image is percieved as 2D matehmatically I think, even though in practise it's 3D because of color channels. These are not flattened as in VAE, where we would have a 1D array for an image and 2D array with batches. The preprocessing step in DDPM seems to be to transform the image data from discrete pixel values $\{0,.., 255\}$ to continuous values $[-1,1]$ and the reason for this is for the model to work with similarly scaled values when starting from the standard gaussian pure noise in the backward process. Note that as a natural consequence of how we define the forward process, the backward process will also be gaussian. Now whether you learn this distribution directly or not will depend on what formulation you choose, we will cover this in the loss section.

### Forward process
**Idea of forward process**
\
The idea of the forward process is usually described as starting with a clean image and destroying it through a series of noisifying steps, where gaussian noise is added to it. Note that for the entire forward process we are only working with continuous values for the data and the noise. How much noise is added at each time step is decided by the scheduler that varies as a variable $\beta \in [0,1]$. It starts off from 0 and increases gradually to 1. The purpose is to eventually reach a state where the image becomes random noise distributed by a standard gaussian $N(0,I)$. For the DDPM they used a linear scheduler starting from $\beta_1 = 10^{-4}$ to $\beta_T = 0.02$. We call each intermediate noisy image a latent $x_t$ at time t. Mathehmatically this process is characterized as a Markov chain, where each latent is only dependent on the previous latent. Illustration of this

![](assets/forward_process.png)

As can be seen each latent is generated only depending on the previous latent $x_t \sim q(x_t|x_{t-1})$. This is the markovian structure for diffusion. And as time goes, the original image gets noiser. Note that timesteps are discrete in DDPM. Another visualization of the process is given by [Strumke and Langseth](https://arxiv.org/pdf/2312.10393).

<img src="assets/forward_process_norwegian.png" style="width: 75%; height: 75%"></img>

We can see that the data distribution transforms gradually to a standard gaussian. The path shown in the image is called a diffusion path. As can be seen, if time is continuous, then there are intersections in the paths, meaning that the two clean images, even with different starting points (being two different images) at some point crossed paths and became the same noisy image, but diverged to end up in different ending points within the standard gaussian distribution. Another observation which is also true, but not shown here, is that the ending points can be different for two images that have the same starting points (being the same image). This demonstrates the nature of the stochasticity in the diffusion process.

**Diffusion kernel - jump steps**
\
However, to make the forward process more efficient, one can bypass this restriction of having to go one step at a time, by using the diffusion kernel to immediately jump to the latent at timestep t, which makes it more efficient to sample latents in the forward process. The diffusion kernel is an important part of the diffusion process and can be described as 

$$x_t = \sqrt{\bar{\alpha}_t}x_0 + \sqrt{1 - \bar{\alpha}_t}\epsilon$$

where $x_t$ is the latent variable (hidden), $\bar{\alpha}_t = \prod_{t=1}^{T}1-\beta_t$, $\beta_t \in [0, 1]$ and $\epsilon \sim \mathcal{N}(0, I)$. In general $\bar{\alpha}_t \to 0$, $1-\bar{\alpha}_t \to 1$ and $\beta_t \to 1$ as $t \to T$, where $\beta_t$ starts from 0. Alternatively one can describe it as a conditional gaussian distribution

$$x_t \sim q(x_t | x_0) = \mathcal{N}(\sqrt{\bar{\alpha}_t}x_0, \sqrt{1 - \bar{\alpha}_t}I)$$

There is a dual interpretation of diffusion kernel as far as I'm concerned 
1. It's a gaussian shift and scaling, since we are adding $\sqrt{\bar{\alpha}_t}x_0$ to the noise that comes from a standard gaussian distribution and scaling it by $\sqrt{1 - \bar{\alpha}_t}$. This also means that the starting image $x_0$ heavily affects the possible positions for the latent $x_t$, which is natural, because the clean image is used as a starting point to add noise to.
2. Scaling the image down and adding noise to it, this is probably the most common interpretation on the internet, at least in introductory blogs/videos about diffusion.

Note that the diffusion kernel is not learnt, it is completly pre-determined given an image $x_0$ and a scheduler. 
Furthermore, if one views the $x_0$ as a random variable distributed by the data distribution, then we can interpret it together with the forward process as gradually transforming a very complex data distribution into a gaussian distribution. Another way of looking at it is to say that the data is transformed from in-distribution to out-distribution, where the in-distribution is the data distribution of the images and out-distribution is anything outside the data distribution, basically meaning that the noisy image is no longer in the data manifold.

![](assets/in_and_out_dist.png)

If we assume the circle to be the data distribution, then adding noise to an image will bring it outside.

**Why scale the image down?**
\
One might wonder where the scalars $\sqrt{\bar{\alpha}_t}$ and $\sqrt{1-\bar{\alpha}_t}$ come from in the diffusion kernel. $\sqrt{1-\bar{\alpha}_t}$ is for scaling up the variance and [Chan](https://arxiv.org/pdf/2403.18103) shows that when $t \to \infty$ then in the limit in order for the variance of the gaussian distribution to be I the scalar has to be precisely $\sqrt{1-\bar{\alpha}_t}$, but the $\sqrt{\bar{\alpha}_t}$ is not explained at all and comes out of nowhere. From what I've seen it should have a variance preserving purpose, to not make the variance explode, but I don't understand it myself. The simplest explanation though is that we want the mean to converge to zero at the end of the forward process, so that's why we need to scale the image down in the diffusion kernel. However, this doesn't still explain why specifically it has to be $\sqrt{\bar{\alpha}_t}$, because it could have been any scalar that can scale down the image. This remains a mystery for now. Ideally a proof would be nice to see.

**Shape of noise**
\
The shape of the noise is the same as the data, which is 2D in the mathematical interpretation, but 4D in practise if we have color channels and a batch. Because we are actually working with multivariate gaussians and the noise is sampled from a standard gaussian with a diagonal covariance matrix, this means that each entry in the 2D shaped noise array can be seen as being sampled from their own univariate gaussian distributions. And after scaling this noise and adding it to the clean image, we can interpret that as we are adding each pixel of the image with noise, where each noise element in the 2D array comes from their own univariate gaussian distribution. Once again, the intuition for why zero covariance (diagonal covariance matrix) implies zero independence only for gaussians can be found in the [cs229 stanford](https://cs229.stanford.edu/notes-spring2019/cs229-notes2.pdf) section 3.

**Why does forward process work?**
\
Intuitively it's a much harder task to go from pure noise to a clean image in one step. Going one step at a time with smaller adjustments to the clean image by adding noise to it, will only change the distribution slightly, whether it's marginal $p(x_t)$ or conditional $p(z_t|z_{t-1})$. The change is much easier to learn than the change from pure noise to data distribution, because each intermediate distribution is similar to the previous one.

**Marginal vs conditional**

> I'm not sure about this particular subsection. Needs to be revised!!!

Earlier we had some discussions on the difference between the marginal and the conditional of the latents. The marginal was brought up, because [Prince chapter 18](https://udlbook.github.io/udlbook/) had visualized the marginals in the forward process. We came to the conclusion that the marginal $q(x_t)$ can be seen as representing the average bluriness in the image at time t, while the conditional $q(z_t|x)$ that is the diffusion kernel, can be seen as the bluriness for a given image x. For the marginal case I think this can be made clearer by observing the following expression

$$E_{p(x)}\left[q(x_t|x)\right] = \int q(x_t|x)p(x) \, dx = \int q(x_t,x) \, dx = q(x_t)$$

where the integral is taken over all possible x, meaning all possible data points. This basically says to compute the average of the latent $x_t$ conditioned on x where x comes from the data distribution. So it's the average bluriness in the image at time t.
However, note that this marginal is different from the marginal likelihood that is the data distribution. This only concerns with the latents and this can be seen if we just look at how it's computed

$$q(x_t) = \int_x q(x_t|x)p(x) \, dx$$

where $q(x_t | x)$ is the diffusion kernel and $p(x)$ is the data distribution, whereas for the marginal data distribution we have

$$p(x) = \int q(x_T) \prod_{t=1}^T q_{\theta}(x_{t-1} | x_t) \, dx_{1:T} = \int q(x_{0:T}) \, dx_{1:T}$$

so it should be clear that they are not the same? 

**Forward process in practise**
\
While many sources on the web make diffusion out to be this markovian chain, where one step is taken at a time to generate the next latent. In practise, the only time the forward process comes into play is during training, but even then it's different. Time steps are uniformly sampled, so we never sequentially iterate from 1 to T. Why sample time steps? No idea, heard it has a regularizing effect to help with generalization and perhaps it relates to Monte carlo, but not sure.

**Latent diffusion model**
\
Lastly, since this has been brought up in the group multiple times, the biggest difference between latent diffusion models such as stable diffusion and DDPM is that there's a VAE before training to downsample the image latent to lower dimensions, which makes it faster to train and infer. After sampling the latents are upsampled to the original image resolution by the decoder in VAE.

### Loss function - noise and other formulations
Deriving the loss function is the hardest part of the theoretical side of diffusion, at least DDPM diffusion. There's alot of algebraic manipulations and mathematical tricks applied. I won't derive anything here, read the suggested papers in the next section if you want to see full derivations, I highly recommend the paper by Calvin Luo 2022. Let's start with the log likelihood data $p(x)$. Because integrating out all the possible latents $\log p(x) = \int p(x_0,...,x_T) \, dx_{1:T}$ is intractable we can instead with clever manipulations arrive at the ELBO. Our job is to maximize this ELBO, so we can approach the data log likelihood $\log p(x)$, we want it as close as possible, because if we can train a model to approximate it well, then samples drawn from it will be representative of the data distribution.

$$\log p(x) \geq \underbrace{\underbrace{E_{q(x_1|x_0)}\left[\log p_{\theta}(x_0|x_1)\right]}_{\text{reconstruction term}} - \underbrace{D_{KL}(q(x_T | x_0) || p(x_T))}_{\text{prior matching term}} - \underbrace{\sum_{t=2}^T D_{KL}(q(x_{t-1}|x_t,x_0)  || p_{\theta}(x_{t-1} | x_t))}_{\text{denoising matching term}}}_{\text{ELBO}}$$

where the reconstruction term is the same as in VAE and can be approximated with Monte carlo. However, later on this term can be ignored because $x_1$ is very similar to $x_0$ if you can make the scheduler have a small gap between timestep 0 and 1, but an alternative explanation that I like better is given by [Prince chapter 18](https://udlbook.github.io/udlbook/), where he shows that you can merge this term together with the denoising matching term. The prior matching term is zero under the assumption that our prior is the standard gaussian, so we can get an exact match. As for the denoising matching term this needs to be massaged a bit more before one can turn it into a loss function. Clever use of Bayes' rule on the backward distribution and evaluating and simplifying the product of gaussians gives the posterior for the backward process distribution

$$q(x_{t-1}|x_t, x_0) \sim \mathcal{N}\left(\underbrace{\dfrac{\sqrt{\alpha_t}(1-\bar{\alpha}_{t-1})x_t + \sqrt{\bar{\alpha}_{t-1}}(1-\alpha_t)\textcolor{green}{x_0}}{1-\bar{\alpha}_t}}_{\mu_q(x_t, x_0)}, \underbrace{\dfrac{(1-\alpha_t)(1-\bar{\alpha}_{t-1})}{1-\bar{\alpha}_t}I}_{\sum_q(t)}\right)$$

Now the biggest take-away from [Luo 2022](https://arxiv.org/pdf/2208.11970) is that at this point reparametrizing $\textcolor{green}{x_0}$ in the mean of this posterior backward distribution will give different objectives/formulations of diffusion, basically resulting in different loss functions for diffusion. To get the noise formulation we need to isolate $x_0$ in the diffusion kernel and substitute the expression into the mean. Simplifying the resulting expression and substituting that into the denoising matching term and applying [KL-divergence for two multivariate gaussians](https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence#Multivariate_normal_distributions) gives the final squared l2 distance of the noise differences leading to the following objective 

$$\operatorname*{argmin}_{\theta} ||\epsilon_0 - \epsilon_{\theta}(x_t, t)||_2^2$$

There is some scaling factor that comes with it as well, but it's ignored to reweight the ELBO. This in combination with the training setup in DDPM leads to down-weighting the earlier timesteps where the noise is small between the steps to make the model focus on harder denoising tasks in later timesteps. In addition to the noise formulation one can reparametrize the mean into 3 additional objectives, which all differ up to a scaling factor.
1. Predict the original image given noisy image at time t
2. Predict the mean of the backward posterior distribution $q(x_{x-1}|x_t)$
3. Predict the score function $\nabla_x \log p(x)$

It's interesting to note that for the noise formulation we are using the mean of the backward posterior distribution as a **proxy** to learn the noise by reparametrizing the mean with $\epsilon_0$ through $x_0$, but in the end what we end up learning is to predict the noise in a noisy image and not the mean of the posterior backward distribution. Furthermore, one could decide to learn the variance by formulating it as of the posterior backward distribution, but DDPM found that gave unstable training and poorer sample quality, so they opted for fixed variances decided by the scheduler. This is the reason why they only look at the mean instead of looking at both the mean and the variance, because naturally one might think to learn all the distribution parameters when wanting to learn a distribution.

**Where is Monte carlo?**
\
I already addressed this in the prequisites under the section Monte carlo, but just to reiterate MC is used in VAE to approximate the decoder loss and since DDPM is a form of markovian hierarchical VAE, then it feels natural that there would also be some MC involved. However, for the reconstruction term $p_{\theta}(x_0|x_1)$ we can merge it into the denoising matching term $D_{KL}(q(x_{t-1}|x_t,x_0)  || p_{\theta}(x_{t-1} | x_t))$ and in turn this can be computed in closed-form by using the formula for [KL divergence between two gaussians](https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence#Multivariate_normal_distributions). The prior term $p(x_T)$ is already assumed to be standard gaussian, so there's no need to compute it, because we already know it. Therefore, no MC is needed, although I speculate that the random sampling of timesteps as well as the mini-batch gradient descent in the training can be seen as some form of MC, but I'm not sure.

### Training
The algorithm for training is straightforward and note that it is the weights of the U-net that are being trained. U-net with self attention is used.

![](assets/training_diffusion.png)

Time is uniformly sampled and each time step doesn't have any weighting, meaning each time step is equally as important. T is set to 1000. The model is learned to predict the noise given a noisy image and the timestep. [Katherine Crowson](https://github.com/crowsonkb) recommends to use bigger batch sizes when training diffusion models, usually 512 or higher, and the reasoning is because the inherent noisy process renders gradients generally noisy as well, so to combat this a bigger batch size is desirable. Although, it should be noted that DDPM used a lesser batch size. This is unsupervised, so no labels are needed and the ground truth is instead the sampled standard gaussian noise at time t. In a sense training the model in the forward process is teaching it to denoise, because noisy_image - noise = clean_image. Lastly, because of noise perturbation to the data, this can be seen as a form of data augmentation, since each data with perturbed noise is a slightly new kind of data point.

### Backward process
Unlike the forward process where we could just use the diffusion kernel to jump t steps, in the backward process we are stuck with the markovian structure. So we need to go one step at a time denoising the sampled standard gaussian noise until we get a clean image. This process is the **inference step** and therefore the sampling process, where we use our trained model to predict the noise at time t given a noisy latent. The process can be described in the following image.

![](assets/backward_process.png)

Note that at the end going from $t=1$ to $t=0$, the continous values have to be converted to discrete pixels, which was initially described by the authors of DDPM to be done by a discrete decoder. The equation for this can be found in the [DDPM paper](https://arxiv.org/pdf/2006.11239) page 4 equation 13. However, later on, they dropped this entirely and instead chose to just not add noise on the last step before creating the clean image.
The backward distribution can be derived with Bayes' rule to be a gaussian.

$$q(x_{t-1}|x_t, x_0) = \dfrac{q(x_t | x_{t-1}, x_0)q(x_{t-1} | x_0)}{q(x_t | x_0)} \propto \mathcal{N}\left(\underbrace{\dfrac{\sqrt{\alpha_t}(1-\bar{\alpha}_{t-1})x_t + \sqrt{\bar{\alpha}_{t-1}}(1-\alpha_t)x_0}{1-\bar{\alpha}_t}}_{\mu_q(x_t, x_0)}, \underbrace{\dfrac{(1-\alpha_t)(1-\bar{\alpha}_{t-1})}{1-\bar{\alpha}_t}I}_{\sum_q(t)}\right)$$

Since we didn't know what the backward distribution was, we used Bayes' rule to flip the condition and therefore could use the forward distribution instead together with diffusion kernels to compute the backward distribution. So we are essentially relying on formulas from the forward process to compute the backward process. 

**Interpretations**
\
One way to interpret the denoising steps from pure noise to clean image is that we are moving the noisy image sample from the out-distribution back into the in-distribution of images.

![](assets/out_to_in_dist.png)

Note that this is the overall effect, but it's not done in one step, usually in hundreds or thousand steps. So we will be creating intermediate noisy images before reaching the clean image at the end. The path of moving from noisy image to less noisy image is called a diffusion path (maybe backward diffusion path). Each point in this diffusion path is a decision point where you can go from one mode to another in the data distribution. This can be nicely illustrated using the score formulation, which is equivalent to the noise formulation up to a scaling factor. The reason I bring it up is because it's very intuitive.

![](assets/diffusion_path_3D.png)
![](assets/vector_field_diffusion_path.png)

Here we have a 3D visualization of the diffusion path in the backward process and the [vector field](https://en.wikipedia.org/wiki/Vector_field) for it taken from [Luo 2022](https://arxiv.org/pdf/2208.11970). So we can see that the diffusion path varies and that at a certain point where the noisy image moves is decided by the gradient of the score function (gradient or derivative of the log data probability $\log p(x)$), which gives the steepest ascent that points in the direction of the peak (mode) of the data probability distribution. For clarification, a vector field is a vector-valued function $f(x_1, ..., x_n): \mathbb{R}^n \mapsto \mathbb{R}^n$, where the dimensionality of the input and output is the same, and the graph is created by drawing out vectors (showing directions) at all possible points in the space, with some normalized length so as to not clutter the graph too much. The vectors in the vector field points in the direction the vector should move for the next step, so we can imagine the images being these vectors and where the denoised image will move is decided by this direction. Because of the extra noise added in the sampling process, this will encourage the clean image to not fully converge to the modes of the data distribution if the diffusion path moves that far, which prevents [mode collapse](https://developers.google.com/machine-learning/gan/problems#mode-collapse), and therefore encouraging diversity in the produced images. Another way of saying this is that diffusion paths that move to the peaks will be encouraged to oscillate around the peak instead of ending exactly at the peak. Note that not all clean images will end up at the peaks, or necessarily near the peaks, it's just that on average they will end up there or in the vicinity.

Another thing to note is that each point in a diffusion path can theoretically be passed by other paths creating intersections, meaning that two different noisy images with different starting points can still pass each other resulting in them being the exact same noisy image at a certain point in the denoising process, but then end up as different clean images, because the stochasticity of the backward process makes this possible. This can be illustrated by the following graph by [Strumke and Langseth](https://arxiv.org/pdf/2312.10393)

<img src="assets/backward_diffusion_path_norwegian.png" style="width: 75%; height: 75%"></img>

Note that this is just 1D data, so it's simplified. We can see that, if time is continuous, there are two intersections near the end between two different diffusion paths. At these intersection they become the exact same noisy image, however diverge into different clean images at the end. Furthermore, two noisy latents with the same starting point can diverge into two different clean images at the end. These two observations are the same ones made for the forward process, but now going backwards. Another way to interpret the backward process is that we are changing the prior standard gaussian, where we start with pure noise, into a complex distribution that is hopefully close to the data distribution. Going from right to left in the graph above, that's the backward process. So here it's clear that we start from a gaussian and then gradually transform this distribution into a more complex distribution, that should be similar to the data distribution. Lastly, the mathematical interpretation of the backward process is that you are basically sampling $x_{t-1}$ from $q_{\theta}(x_{t-1}|x_t)$, that is $x_{t-1} \sim q_{\theta}(x_{t-1}|x_t)$. This happens at each step and we are always dependent on the previous latent or sample in order to compute the next sample because of the markovian structure.

**Deriving the sample algorithm**
\
The sample algorithm looks like this taken from the DDPM paper.

![](assets/backward_sampling.png)

And it's pretty straightforward, we follow the sampling algorithm until the end of the loop at which point we don't add noise to the latents, because at that point we have a clean image that we don't want to perturb. Now where does this sampling formula come from? It comes from the posterior backward distribution that we showed at the beginning of this section! If we rewrite it as a gaussian shift and scale then we get the formula.

$$
\begin{align}
x_{t-1} \sim q_{\theta}(x_{t-1} | x_t) 
&= \mathcal{N}(x_{t-1}|\mu_{\theta}(x_t), \sigma_q^2(t)\textbf{I}) \\
&= \mu_{\theta}(x_t) + \sigma_q^2(t)z \\
&= \dfrac{\sqrt{\alpha_t}(1-\bar{\alpha}_{t-1})x_t + \sqrt{\bar{\alpha}_{t-1}}(1-\alpha_t)x_0}{1-\bar{\alpha}_t} + \sigma_q^2(t)z \\
&= \left\{\text{Reparametrize with epsilon and simplify}\right\} \\
&= \dfrac{1}{\sqrt{\alpha_t}}\left(x_t - \dfrac{1-\alpha_t}{\sqrt{1-\bar{\alpha}_t}}\epsilon_{\theta}(x_t)\right) + \sigma_q^2(t)z
\end{align}$$

where $\alpha_t = 1 - \beta_t$, $\sigma_q^2(t) = \dfrac{(1-\alpha_t)(1-\bar{\alpha}_{t-1})}{1-\bar{\alpha}_t}$, $z \sim \mathcal{N}(0, I)$. Note that in this notebook I prefer to denote the standard gaussian noise as $\epsilon$, but the algorithm in DDPM uses z as notation and for pedagogical reason I opted for z in the derivation above. Anyways, the simplifcation is pure algebraic manipulation, where $x_0$ is reparametrized as $\epsilon_{\theta}$. The full derivation steps can be found in [Calvin Luo 2022 page 15](https://arxiv.org/pdf/2208.11970). But that's all there is to it.

### Other things
* Sampler and scheduler is different! Some people use them interchangeably but they are not the same thing!

**Noise scheduler**
\

**Samplers**
\
* diffusion backward scheduler only takes a small proportion of the predicted noise, so we can move slowly from T to t=0. Each timestep will produce a latent that is slowly denoised

**Architecture**
\

**Conclusion**
\
So just to recap, we have done...


## Formal walkthrough
After giving this section some thought, I don't think I'm able to provide anything of value than what hasn't already been shown in other papers and books. There are three good papers and a book on the diffusion math that covers it from scratch
1. [Understanding Diffusion Models: A Unified Perspective](https://arxiv.org/pdf/2208.11970). I like this one alot because it shows literally all the derivation steps. It also shows different objectives of diffusion that all are equal up to a scaling factor. The objectives are mean of posterior for the reverse process, noise in images in the forward process, original image $x_0$ given arbitrary latent $x_t$ and the score function.
2. [Tutorial on Diffusion Models for Imaging and Vision](https://arxiv.org/pdf/2403.18103). Also from scratch, but elaborates on the score function formulation more so than the paper above. Also goes into ODE and SDE of diffusion.
3. [Step-by-Step Diffusion: An Elementary Tutorial](https://arxiv.org/pdf/2406.08929). Also from scratch, but covers it from a different perspective. Covers DDPM and DDIM. Also covers flow matching, which is a neat technique to know since stable diffusion 3 uses it.
4. [Understanding deep learning](https://udlbook.github.io/udlbook/). Chapter 18 covers diffusion models, specifically DDPM using the noise objective, i.e. model learns to predict the noise added to latents. Good breakdown, although two of the formulas used, change of variable in gaussian and product of two gaussians, can feel a bit mysterious. The first paper listed here does give a good derivation of product of two gaussians without using the formula for product of two gaussians, so look at the paper if it's not clear.
5. [Original DDPM paper](https://arxiv.org/pdf/2006.11239). Then of course reading the original paper is always good, although they don't usually have the space to explain all the steps or show all the derivations, so it's usually more compressed.